Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [186]:
from random import choice
from tqdm.auto import tqdm
import numpy as np
from heapq import heappush, heappop
from collections import namedtuple
from typing import List
import heapq

In [187]:
# Define the Action type
Action = namedtuple('Action', ['pos1', 'pos2'])

# Define puzzle dimension
PUZZLE_DIM = 7

In [188]:
def available_actions(state: np.ndarray) -> List[Action]:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = []
    if x > 0:
        actions.append(Action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(Action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(Action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(Action((x, y), (x, y + 1)))
    return actions

"""Apply the action to the state."""
def do_action(state: np.ndarray, action: Action) -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state



In [189]:
def manhattan_distance(state: np.ndarray, goal: np.ndarray) -> int:
    """Calcola la distanza di Manhattan tra lo stato attuale e quello finale."""
    distance = 0
    for num in range(1, PUZZLE_DIM**2):  # Ignoriamo 0 (lo spazio vuoto)
        x1, y1 = np.where(state == num)
        x2, y2 = np.where(goal == num)
        distance += abs(x1[0] - x2[0]) + abs(y1[0] - y2[0])
    return distance


In [190]:
def a_star(start_state: np.ndarray, goal_state: np.ndarray) -> List[np.ndarray]:
    """Solve the puzzle using A*."""
    open_set = []
    # Convert start state to tuple for immutability and comparison in sets
    start_state_tuple = tuple(start_state.flatten())
    goal_state_tuple = tuple(goal_state.flatten())

    distance = manhattan_distance(start_state, goal_state)
    heappush(open_set, (distance, 0, start_state_tuple, []))

    visited = set()
    visited.add(start_state_tuple)

    while open_set:
        f, g, current_state_tuple, path = heappop(open_set)
        current_state = np.array(current_state_tuple).reshape((PUZZLE_DIM, PUZZLE_DIM))

        # Goal test
        if np.array_equal(current_state, goal_state):
            return path + [current_state]

        # Generate successors
        for act in available_actions(current_state):
            successor = do_action(current_state, act)
            successor_tuple = tuple(successor.flatten())
            if successor_tuple not in visited:
                new_path = path + [current_state]  # Copy and append current state
                new_cost = g + 1
                heappush(open_set, (new_cost + manhattan_distance(successor, goal_state), new_cost, successor_tuple, new_path))
                visited.add(successor_tuple)

    return None  # No solution found

In [191]:
def greedy_search(start: np.ndarray, goal: np.ndarray) -> List[np.ndarray]:
    """Implementazione dell'algoritmo di ricerca Greedy usando la distanza di Manhattan."""
    # Coda di priorità per esplorare gli stati più promettenti (minore distanza)
    frontier = []
    heapq.heappush(frontier, (manhattan_distance(start, goal), start))  # (f, stato)

    explored = set()  # Insieme degli stati esplorati
    start_tuple = tuple(start.flatten())
    explored.add(start_tuple)  # Aggiungi lo stato iniziale esplorato come tupla

    parent_map = {start_tuple: None}  # Per ricostruire il percorso

    while frontier:
        # Prendi il nodo con la minima distanza di Manhattan
        _, current_state_tuple = heapq.heappop(frontier)
        current_state = np.array(current_state_tuple).reshape((PUZZLE_DIM, PUZZLE_DIM))

        # Se abbiamo raggiunto il goal, ricostruiamo il percorso
        if np.array_equal(current_state, goal):
            path = []
            while current_state is not None:
                path.append(current_state)
                current_state = parent_map[tuple(current_state.flatten())]
            return path[::-1]  # Ritorna il percorso dal start al goal

        # Esplora i successori
        for action in available_actions(current_state):
            new_state = do_action(current_state, action)
            new_state_tuple = tuple(new_state.flatten())
            if new_state_tuple not in explored:
                explored.add(new_state_tuple)  # Marca come esplorato
                heapq.heappush(frontier, (manhattan_distance(new_state, goal), new_state_tuple))
                parent_map[new_state_tuple] = current_state  # Salva il predecessore

    return None  # Nessuna soluzione trovata


In [192]:
Goal_State = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

RANDOMIZE_STEPS = 100 #100_000  # 100 steps
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
np.random.seed(42)  # For reproducibility

for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))

print("Initial State:")
print(state)

#solution_path = a_star(state, Goal_State)
solution_path=greedy_search(state, Goal_State)
print("\nSolution Steps:")
for step in solution_path:
    print(step)

Randomizing: 100%|██████████| 100/100 [00:00<00:00, 49902.49it/s]


Initial State:
[[ 1  2  3  4 12  6  7]
 [ 9 10 17  5 11 13 14]
 [ 8 16 24 18 19 20 21]
 [22  0 25 31 26 27 28]
 [15 29 23 30 32 34 35]
 [36 37 38 46 33 40 48]
 [43 44 45 47 39 42 41]]

Solution Steps:
[[ 1  2  3  4 12  6  7]
 [ 9 10 17  5 11 13 14]
 [ 8 16 24 18 19 20 21]
 [22  0 25 31 26 27 28]
 [15 29 23 30 32 34 35]
 [36 37 38 46 33 40 48]
 [43 44 45 47 39 42 41]]
[[ 1  2  3  4 12  6  7]
 [ 9 10 17  5 11 13 14]
 [ 8 16 24 18 19 20 21]
 [ 0 22 25 31 26 27 28]
 [15 29 23 30 32 34 35]
 [36 37 38 46 33 40 48]
 [43 44 45 47 39 42 41]]
[[ 1  2  3  4 12  6  7]
 [ 9 10 17  5 11 13 14]
 [ 8 16 24 18 19 20 21]
 [15 22 25 31 26 27 28]
 [ 0 29 23 30 32 34 35]
 [36 37 38 46 33 40 48]
 [43 44 45 47 39 42 41]]
[[ 1  2  3  4 12  6  7]
 [ 9 10 17  5 11 13 14]
 [ 8 16 24 18 19 20 21]
 [15 22 25 31 26 27 28]
 [29  0 23 30 32 34 35]
 [36 37 38 46 33 40 48]
 [43 44 45 47 39 42 41]]
[[ 1  2  3  4 12  6  7]
 [ 9 10 17  5 11 13 14]
 [ 8 16 24 18 19 20 21]
 [15 22 25 31 26 27 28]
 [29 23  0 30 32 34 35]
 [3